# Dealing with imbalanced datasets, combining oversampling with VAE and undersampling to improve model recognition over all classes.  

In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.6 MB/s eta 0:00:00


Import packages, classifiers and etc.

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

from sklearn.metrics import confusion_matrix, make_scorer
from imblearn.metrics import classification_report_imbalanced, geometric_mean_score
from collections import Counter

from imblearn.over_sampling import SMOTE, RandomOverSampler

Import VAEOversampler.

In [3]:
from VAEOversampler import VAEOversampler

## Loading data  
You can load some dataset from Imbalanced Learn list (https://imbalanced-learn.org/stable/datasets/index.html) or use your own data.  


In [4]:
from imblearn.datasets import fetch_datasets

dset_name = 'arrhythmia'
dset = fetch_datasets()[dset_name]

X, y = StandardScaler().fit_transform(dset.data), dset.target

In [5]:
X

array([[ 1.73443926, -1.1075202 ,  0.64132669, ...,  1.17773685,
         0.29460309,  1.07867028],
       [ 0.57931213,  0.90291807, -0.03199781, ...,  0.61612582,
         0.0796127 ,  0.50487408],
       [ 0.4577198 , -1.1075202 ,  0.15653305, ...,  1.52874375,
        -0.52087767,  1.0570176 ],
       ...,
       [-0.63661117, -1.1075202 , -0.00506483, ..., -0.15608935,
        -4.7094834 , -3.3926096 ],
       [-0.87979583,  0.90291807, -0.30132761, ...,  0.82672996,
         0.42063193,  0.92710147],
       [ 1.91682776,  0.90291807, -0.16666271, ...,  0.26511892,
         0.14633386,  0.18008377]])

In [6]:
y[y == -1] = 0

We split data into train and test partitions.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

This is a simple function to undersample freely.  

In [8]:
# RUS

def RUS(X_res, y_res, frac=1, minority_class_id=1, random_state=42):
    X_res = pd.DataFrame(X_res)
    X_res['Class'] = y_res

    X_neg = X_res[y_res != minority_class_id].sample(frac=frac, random_state=random_state)
    X_pos = X_res[y_res == minority_class_id].sample(frac=1, random_state=random_state)

    X_rus = pd.concat([X_neg, X_pos], ignore_index=True)

    X_eq = X_rus.drop('Class', axis=1)
    y_eq = X_rus['Class']

    return X_eq, y_eq

In [9]:
def train_val(X, y, Xt, yt, random_state=42):
    classifiers = {
        "CatBoostClassifier": CatBoostClassifier(verbose=False, random_seed=random_state),
        "LGBMClassifier": LGBMClassifier(random_state=random_state),
        "XGBClassifier": XGBClassifier(random_state=random_state),
        "BaggingClassifier": BaggingClassifier(random_state=random_state),
        "RandomForestClassifier": RandomForestClassifier(random_state=random_state),
    }
    scores = []
    predictions = []
    for key, classifier in classifiers.items():
        print('_' * 50)
        name = key
        classifier.fit(X, y)
        print("Classifier: ", name)
        y_pred = classifier.predict(Xt)
        cm = confusion_matrix(yt, y_pred)
        print(cm)
        print('')
        predictions.append(y_pred)
        tn = cm[0,0]
        fp = cm[0,1]
        fn = cm[1,0]
        tp = cm[1,1]
        tnr = tn / (tn + fp)
        tpr = tp / (tp + fn)
        scores.append(tnr * tpr)
        print('TNR:', round(tnr, 5))
        print('TPR:', round(tpr, 5))
        print('TNRxTPR:', round(tnr * tpr, 5))
        print('G-mean:', round(np.sqrt(tnr * tpr), 5))

    print('_' * 50)
    print('Ensemble predictions (majority voting):')
    predictions = np.sum(predictions, axis=0)
    predictions[predictions < 3] = 0
    predictions[predictions >= 3] = 1

    cm = confusion_matrix(yt, predictions)
    print(cm)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    tnr = tn / (tn + fp)
    tpr = tp / (tp + fn)
    print('')
    print('TNR:', round(tnr, 5))
    print('TPR:', round(tpr, 5))
    print('TNRxTPR:', round(tnr * tpr, 5))
    print('G-mean:', round(np.sqrt(tnr * tpr), 5))


## Without resampling (base line)
Which is starting score?


In [10]:
print('Original dataset shape %s' % Counter(y))
print('Ratio->', round(Counter(y)[0]/Counter(y)[1], 1), ': 1')

Original dataset shape Counter({0: 427, 1: 25})
Ratio-> 17.1 : 1


In [11]:
train_val(X_train, y_train, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 3  2]]

TNR: 1.0
TPR: 0.4
TNRxTPR: 0.4
G-mean: 0.63246
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  BaggingClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 5  0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


## Ratio 1:1  
Let's see classifiers scores when dataset is balanced.  


In [12]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [13]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 341, 1: 341})
Ratio->  1 : 1.0


In [14]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 4  1]]

TNR: 1.0
TPR: 0.2
TNRxTPR: 0.2
G-mean: 0.44721
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


In [15]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [16]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0: 341, 1: 341})
Ratio->  1 : 1.0


In [17]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 3  2]]

TNR: 1.0
TPR: 0.4
TNRxTPR: 0.4
G-mean: 0.63246
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  BaggingClassifier
[[84  2]
 [ 1  4]]

TNR: 0.97674
TPR: 0.8
TNRxTPR: 0.7814
G-mean: 0.88397
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 4  1]]

TNR: 1.0
TPR: 0.2
TNRxTPR: 0.2
G-mean: 0.44721
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0


In [18]:
# VAEOversampler

vae_sampler = VAEOversampler(epochs=500,
                              intermediate_dim=512,
                              batch_size=64,
                              random_state=42,
                              verbose=False)
X_res, y_res = vae_sampler.fit_resample(X_train, y_train)

11/11 [==============================] - 0s 2ms/step


In [19]:
print('Resampled dataset shape %s' % Counter(y_res))
print('Ratio->  1 :', round(Counter(y_res)[1]/Counter(y_res)[0], 1))

Resampled dataset shape Counter({0.0: 341, 1.0: 341})
Ratio->  1 : 1.0


In [20]:
train_val(X_res, y_res, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 3  2]]

TNR: 1.0
TPR: 0.4
TNRxTPR: 0.4
G-mean: 0.63246
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  BaggingClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 5  0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 3  2]]

TNR: 1.0
TPR: 0.4
TNRxTPR: 0.4
G-mean: 0.63246


## Under/Oversampling combination  
Now we can tuning the number of instances for each class to optimize metric.  


In [26]:
# SMOTE

sm = SMOTE(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = sm.fit_resample(X_train, y_train)

In [27]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.9)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 341, 0: 307})
Ratio->  1 : 1.1


In [28]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  BaggingClassifier
[[85  1]
 [ 0  5]]

TNR: 0.98837
TPR: 1.0
TNRxTPR: 0.98837
G-mean: 0.99417
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 4  1]]

TNR: 1.0
TPR: 0.2
TNRxTPR: 0.2
G-mean: 0.44721
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


In [29]:
# ROS

ros = RandomOverSampler(random_state=42, sampling_strategy=1) # N_rm / N_M
X_res, y_res = ros.fit_resample(X_train, y_train)

In [30]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.8)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1: 341, 0: 273})
Ratio->  1 : 1.2


In [31]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 4  1]]

TNR: 1.0
TPR: 0.2
TNRxTPR: 0.2
G-mean: 0.44721
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443


In [32]:
# VAEOversampler

X_res, y_res = vae_sampler.resample(X_train, y_train, sampling_strategy=1)

11/11 [==============================] - 0s 2ms/step


In [35]:
# RUS

X_eq, y_eq = RUS(X_res, y_res, frac=.6)

print('Resampled dataset shape %s' % Counter(y_eq))
print('Ratio->  1 :', round(Counter(y_eq)[1]/Counter(y_eq)[0], 1))

Resampled dataset shape Counter({1.0: 341, 0.0: 205})
Ratio->  1 : 1.7


In [36]:
train_val(X_eq, y_eq, X_test, y_test)

__________________________________________________
Classifier:  CatBoostClassifier
[[86  0]
 [ 0  5]]

TNR: 1.0
TPR: 1.0
TNRxTPR: 1.0
G-mean: 1.0
__________________________________________________
Classifier:  LGBMClassifier
[[86  0]
 [ 3  2]]

TNR: 1.0
TPR: 0.4
TNRxTPR: 0.4
G-mean: 0.63246
__________________________________________________
Classifier:  XGBClassifier
[[86  0]
 [ 1  4]]

TNR: 1.0
TPR: 0.8
TNRxTPR: 0.8
G-mean: 0.89443
__________________________________________________
Classifier:  BaggingClassifier
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746
__________________________________________________
Classifier:  RandomForestClassifier
[[86  0]
 [ 5  0]]

TNR: 1.0
TPR: 0.0
TNRxTPR: 0.0
G-mean: 0.0
__________________________________________________
Ensemble predictions (majority voting):
[[86  0]
 [ 2  3]]

TNR: 1.0
TPR: 0.6
TNRxTPR: 0.6
G-mean: 0.7746


## References  

  - Classification with Imbalanced Datasets:  
    https://sci2s.ugr.es/imbalanced  
  - Computer Vision:  Models, Learning, and Inference (Simon J.D. Prince):  
    http://www.computervisionmodels.com/  
  - Oversampling with VAEs:  
    https://towardsdatascience.com/oversampling-with-vaes-e410887fe51  
